In [ ]:
# Environment
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import sys
# Local modules
sys.path.append('..')
import reproducibility
from utils import print_dict_summary
from data_funcs import load_and_fix_data, rmse
from moisture_rnn import create_rnn_data_1, create_rnn_data_2, train_rnn

## Train Model

In [ ]:
reproducibility_file='data/reproducibility_dict.pickle'

repro={}
repro.update(load_and_fix_data(reproducibility_file))
print_dict_summary(repro)

In [ ]:
param_sets_ORIG = {'id':0,
        'purpose':'reproducibility',
        'batch_size':np.inf,
        'training':None,
        'cases':['case11'],
        'scale':0,
        'rain_do':False,
#        'verbose':False,
        'verbose':1,
        'timesteps':5,
        'activation':['linear','linear'],
        'centering':[0.0,0.0],
        'hidden_units':6,
        'dense_units':1,
        'dense_layers':1,
        'DeltaE':[0,-1],    # -1.0 is to correct E bias but put at the end
        'synthetic':False,  # run also synthetic cases
        'T1': 0.1,          # 1/fuel class (10)
        'fm_raise_vs_rain': 2.0,         # fm increase per mm rain                              
        'epochs':5000,
        'verbose_fit':0,
        'verbose_weights':False,
        'note':'check 5 should give zero error'
        }

In [ ]:
case_data = repro["case11"]
h2=case_data['h2']
params = param_sets_ORIG
reproducibility.set_seed() # Set seed for reproducibility
rnn_dat = create_rnn_data_1(case_data,params)
create_rnn_data_2(rnn_dat,params)

print(rnn_dat["x_train"].shape)
print(rnn_dat["y_train"].shape)

In [ ]:
model_predict = train_rnn(
    rnn_dat,
    params,
    rnn_dat['hours'],
    fit=True
)

## Format Data to Predict

In [ ]:
# Destination URL for data files
url = "https://demo.openwfm.org/web/data/fmda/tif/20240101/"

# Get List of files for model with just E's
# Need temp and RH band nums from https://www.nco.ncep.noaa.gov/pmb/products/hrrr/hrrr.t00z.wrfprsf00.grib2.shtml
bands = [616, 620] # temp, RH

# List of hours to predict
pred_hours = [0, 1, 2]

# Format tif files
files = []
for h in pred_hours:
    hr = str(h).zfill(2)
    for b in bands:
        f = f"{url}hrrr.t{hr}z.wrfprsf00.{b}.tif"
        files.append(f)
        print(f"Filename: {f}")

In [ ]:
# Ed = 0.924*rh**0.679 + 0.000499*np.exp(0.1*rh) + 0.18*(21.1 + 273.15 - t2)*(1 - np.exp(-0.115*rh))
# Ew = 0.618*rh**0.753 + 0.000454*np.exp(0.1*rh) + 0.18*(21.1 + 273.15 - t2)*(1 - np.exp(-0.115*rh))

In [ ]:
import rioxarray

In [ ]:
files[0]

In [ ]:
temp = rioxarray.open_rasterio(files[0])

In [ ]:
temp

In [ ]:
plt.imshow(temp.sel(band=1))

In [ ]:
rh = rioxarray.open_rasterio(files[1])
plt.imshow(rh.sel(band=1))

In [ ]:
Ed = 0.924*rh**0.679 + 0.000499*np.exp(0.1*rh) + 0.18*(21.1 + 273.15 - temp)*(1 - np.exp(-0.115*rh))
Ew = 0.618*rh**0.753 + 0.000454*np.exp(0.1*rh) + 0.18*(21.1 + 273.15 - temp)*(1 - np.exp(-0.115*rh))

In [ ]:
plt.imshow(Ed.sel(band=1))
plt.title("Drying Equilibrium")

In [ ]:
plt.imshow(Ew.sel(band=1))
plt.title("Wetting Equilibrium")

In [ ]:
plt.imshow(rh.sel(band=1))

In [ ]:
Ed

In [ ]:
def get_eq_from_url(files):

    # Get right bands
    tfile = [file for file in files if ".616.tif" in file]
    rhfile = [file for file in files if ".620.tif" in file]

    # Data checks
    assert len(tfile) == 1, "More than 1 file found with band 620 (rh), this func only processes 1hr"
    assert len(rhfile) == 1, "More than 1 file found with band 616 (temp), this func only processes 1hr"

    # Read Data
    temp = rioxarray.open_rasterio(tfile)
    rh = rioxarray.open_rasterio(rhfile)
    assert temp.data.shape == rh.data.shape, "Temp and RH data different shapes"

    # Convert C to K if C detected, check is whether any value is less than 150 deg. TODO: do this w metadata
    if np.any(temp < 150):
        temp += 273.15
    
    Ed = 0.924*rh**0.679 + 0.000499*np.exp(0.1*rh) + 0.18*(21.1 + 273.15 - temp)*(1 - np.exp(-0.115*rh))
    Ew = 0.618*rh**0.753 + 0.000454*np.exp(0.1*rh) + 0.18*(21.1 + 273.15 - temp)*(1 - np.exp(-0.115*rh))

    return Ed, Ew